In [1]:
import pandas as pd
import datetime
from datetime import timedelta
from math import radians, cos, sin, asin, sqrt
import numpy as np
import scipy.sparse as spa
import cvxpy as cp
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx

In [2]:
full_data = pd.read_csv("yellow_tripdata_2010-01.csv")
full_data['pickup_datetime'] = pd.to_datetime(full_data['pickup_datetime'])
full_data['dropoff_datetime'] = pd.to_datetime(full_data['dropoff_datetime'])

year = 2010
month = 1
day = 21
date = datetime.date(year, month, day)
midnight = datetime.datetime(year, month, day, 0, 0)

day_data = full_data.loc[(full_data['pickup_datetime'].dt.date == date) & (full_data['dropoff_datetime'].dt.date == date)]
day_data = day_data[['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount']]
day_data = day_data.loc[(day_data['pickup_longitude'] != 0) & (day_data['pickup_latitude'] != 0)]

n = 200
data = day_data.sample(n).sort_values(by = 'pickup_datetime')
data.index = range(n)
data

C:\Users\holly\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pickup_datetime,dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2010-01-21 00:38:08,2010-01-21 00:45:19,1.80,-73.981146,40.729366,-73.971382,40.747983,6.9,0.5,0.5,1.00,0.0,8.90
1,2010-01-21 00:39:04,2010-01-21 00:42:39,0.80,-73.970981,40.755601,-73.978874,40.762910,4.5,0.5,0.5,0.00,0.0,5.50
2,2010-01-21 00:53:38,2010-01-21 00:57:01,0.60,-73.992672,40.725497,-73.984688,40.727443,3.7,0.5,0.5,0.00,0.0,4.70
3,2010-01-21 01:05:00,2010-01-21 01:20:00,4.16,-73.991672,40.721812,-73.961963,40.673393,13.3,0.5,0.5,0.00,0.0,14.30
4,2010-01-21 01:22:04,2010-01-21 01:30:13,1.80,-74.003617,40.750764,-73.988769,40.736792,7.3,0.5,0.5,0.00,0.0,8.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2010-01-21 23:26:33,2010-01-21 23:43:28,3.20,-74.015017,40.709012,-73.985290,40.727752,12.1,0.5,0.5,2.62,0.0,15.72
196,2010-01-21 23:27:07,2010-01-21 23:40:12,7.80,-74.005321,40.715093,-73.960000,40.766809,18.9,0.5,0.5,0.00,0.0,19.90
197,2010-01-21 23:37:00,2010-01-21 23:47:00,2.51,-73.992162,40.749020,-73.976913,40.725580,8.5,0.5,0.5,0.00,0.0,9.50
198,2010-01-21 23:43:00,2010-01-21 23:53:00,1.71,-73.983792,40.743863,-73.997565,40.759420,7.7,0.5,0.5,0.00,0.0,8.70


In [3]:
def to_td(minutes):
    td = []
    for m in minutes:
        td.append(timedelta(minutes = m))
    return td

def to_minutes(td):
    return td.total_seconds() / 60

def T(i,j):
    if (i == j or i == n+taxis): return 0
    # From customer to sink
    if (j == n+taxis+1): return -9999999
        # dist_between = haversine(data.loc[i, 'pickup_longitude'], data.loc[i, 'pickup_latitude'], taxi_lon, taxi_lat)
        # time_between = timedelta(hours = dist_between / mph)
        # output = data.loc[i, 'dropoff_datetime'] - data.loc[i, 'pickup_datetime'] + time_between
    # From taxi to customer
    if (i >= n):
        dist_between = haversine(taxi_lon, taxi_lat, data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
        output = timedelta(hours = dist_between / mph)
    # From customer to customer
    else:
        dist_between = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'],
                                  data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
        time_between = timedelta(hours = dist_between / mph)
        output = data.loc[i, 'dropoff_datetime'] - data.loc[i, 'pickup_datetime'] + time_between
    return to_minutes(output)

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 3956
    return c * r

## Vector Formulation Without Time Window Constraints

In [4]:
taxis = 2            # Number of taxis
taxi_lon = -73.9772  # Initial longitude of taxis
taxi_lat = 40.7527   # Initial latitude of taxis
mph = 20             # Taxi speed between passengers
driving_cost = .6    # in dollars per mile

possible = np.zeros((n+taxis+2, n+taxis+2))  # =1 if connection (i,j) is possible, =0 if impossible
profit_accept = np.zeros((n+taxis+2, n+taxis+2))   # Net profit from accepting passenger j (based on preceding passenger i)
                                         # Note: =0 if connection (i,j) is impossible
source = n+taxis
sink = n+taxis+1
# taxi_cost = 100  # Cost of deploying a taxi in dollars
    
for i in range(n):
    for j in range(n):
        if (i == j): continue

        dist_between = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'],
                                      data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
        time_between = timedelta(hours = dist_between / mph)
        if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime']): continue
        else: possible[i,j] = 1
        profit_accept[i,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])

    possible[i, sink] = 1
#     dist_between = haversine(data.loc[i, 'dropoff_longitude'], data.loc[i, 'dropoff_latitude'], taxi_lon, taxi_lat)
#     profit_accept[i, sink] = -driving_cost * dist_between

for k in range(n, n+taxis):
    for j in range(n):

        dist_between = haversine(taxi_lon, taxi_lat, data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
        time_between = timedelta(hours = dist_between / mph)
        if (midnight + time_between > data.loc[j, 'pickup_datetime']): continue
        else: possible[k,j] = 1
        profit_accept[k,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])

    possible[source, k] = 1
#     profit_accept[source, k] = -taxi_cost

s_list = []
p_list = []

arcs = {}
num_arcs = np.count_nonzero(possible)
A_in = spa.dok_matrix((n+taxis+2, num_arcs))
A_out = spa.dok_matrix((n+taxis+2, num_arcs))
cost = np.zeros(num_arcs)

a = 0
for i in range(n+taxis+2):

    successors = []
    predecessors = []

    for j in range(n+taxis+2):
        if (possible[i,j] == 1):
            successors.append(j)
            if (j > i):
                arcs[(i,j)] = a
                A_in[i, a] = 1
                A_out[j, a] = 1
                cost[a] = profit_accept[i,j]
                a += 1

        if (possible[j,i] == 1):
            predecessors.append(j)
            if (j > i):
                arcs[(j,i)] = a
                A_in[j, a] = 1
                A_out[i, a] = 1
                cost[a] = profit_accept[j,i]
                a += 1

    s_list.append(successors)
    p_list.append(predecessors)

A = A_in - A_out
e = np.zeros(n+taxis+2)
e[source] = 1
e[sink] = -1

# Maximum Flow Problem CVXPY
x = cp.Variable(num_arcs, boolean = True)
t = cp.Variable(n)

## Network Flow Constraints
constraints = [A @ x == taxis * e]
inflow = A_in @ x
constraints += [inflow[:n+taxis] <= 1]

profit = np.transpose(cost) @ x
objective = cp.Maximize(profit)
problem = cp.Problem(objective, constraints)

sims = 100
run_time = 0
for i in range(sims):
    # start timer
    problem.solve()
    # end timer
    run_time += problem.solver_stats.solve_time
print("Run Time:", run_time/sims)
print("Objective Value:", objective.value)

x_val = spa.dok_matrix((n+taxis+2, n+taxis+2))
for i in range(n+taxis+2):
    for j in s_list[i]:
        x_val[i,j] = x[arcs.get((i,j))].value
print(x_val.tocoo())

Using license file C:\Users\holly\gurobi.lic
Academic license - for non-commercial use only
Run Time: 0.4181497001647949
Objective Value: 828.5950769919393
  (0, 2)	1.0
  (1, 4)	1.0
  (2, 3)	1.0
  (3, 8)	1.0
  (4, 5)	1.0
  (5, 6)	1.0
  (6, 10)	1.0
  (8, 9)	1.0
  (9, 11)	1.0
  (10, 16)	1.0
  (11, 12)	1.0
  (12, 13)	1.0
  (13, 15)	1.0
  (15, 24)	1.0
  (16, 25)	1.0
  (24, 31)	1.0
  (25, 28)	1.0
  (28, 34)	1.0
  (31, 35)	1.0
  (34, 36)	1.0
  (35, 37)	1.0
  (36, 42)	1.0
  (37, 41)	1.0
  (41, 44)	1.0
  (42, 46)	1.0
  :	:
  (136, 145)	1.0
  (138, 147)	1.0
  (145, 151)	1.0
  (147, 152)	1.0
  (151, 166)	1.0
  (152, 154)	1.0
  (154, 165)	1.0
  (165, 170)	1.0
  (166, 167)	1.0
  (167, 172)	1.0
  (170, 178)	1.0
  (172, 176)	1.0
  (176, 179)	1.0
  (178, 180)	1.0
  (179, 184)	1.0
  (180, 191)	1.0
  (184, 189)	1.0
  (189, 195)	1.0
  (191, 203)	1.0
  (195, 199)	1.0
  (199, 203)	1.0
  (200, 1)	1.0
  (201, 0)	1.0
  (202, 200)	1.0
  (202, 201)	1.0


## Vector Formulation With Time Window Constraints

In [10]:
window_mins = [0]
time_windows = to_td(window_mins)
run_times = []
objectives = []

taxis = 2            # Number of taxis
taxi_lon = -73.9772  # Initial longitude of taxis
taxi_lat = 40.7527   # Initial latitude of taxis
mph = 20             # Taxi speed between passengers
driving_cost = .6    # in dollars per mile

possible = np.zeros((n+taxis+2, n+taxis+2))  # =1 if connection (i,j) is possible, =0 if impossible
profit_accept = np.zeros((n+taxis+2, n+taxis+2))   # Net profit from accepting passenger j (based on preceding passenger i)
                                         # Note: =0 if connection (i,j) is impossible
source = n+taxis
sink = n+taxis+1
# taxi_cost = 100  # Cost of deploying a taxi in dollars

for time_window in time_windows:
    
    for i in range(n):
        for j in range(n):
            if (i == j): continue

            dist_between = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'],
                                          data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
            time_between = timedelta(hours = dist_between / mph)
            if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime'] + time_window): continue
            else: possible[i,j] = 1
            profit_accept[i,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])
        
        possible[i, sink] = 1
#         dist_between = haversine(data.loc[i, 'dropoff_longitude'], data.loc[i, 'dropoff_latitude'], taxi_lon, taxi_lat)
#         profit_accept[i, sink] = -driving_cost * dist_between

    for k in range(n, n+taxis):
        for j in range(n):

            dist_between = haversine(taxi_lon, taxi_lat, data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
            time_between = timedelta(hours = dist_between / mph)
            if (midnight + time_between > data.loc[j, 'pickup_datetime'] + time_window): continue
            else: possible[k,j] = 1
            profit_accept[k,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])
        
        possible[source, k] = 1
#         profit_accept[source, k] = -taxi_cost
        
    t_min = []
    t_max = []
        
    for i in range(n):
        t_min.append(to_minutes(data.loc[i, 'pickup_datetime'] - midnight))
        t_max.append(to_minutes(data.loc[i, 'pickup_datetime'] - midnight + time_window))
    for k in range(n, n+taxis+2):
        t_min.append(0)
        t_max.append(0)
    
    s_list = []
    p_list = []
    
    arcs = {}
    num_arcs = np.count_nonzero(possible)
    A_in = spa.dok_matrix((n+taxis+2, num_arcs))
    A_out = spa.dok_matrix((n+taxis+2, num_arcs))
    cost = np.zeros(num_arcs)
    
    # For time window constraints
    B = spa.dok_matrix((num_arcs, num_arcs))
    d = np.zeros(num_arcs)
    
    a = 0
    for i in range(n+taxis+2):

        successors = []
        predecessors = []

        for j in range(n+taxis+2):
            if (possible[i,j] == 1):
                successors.append(j)
                if (j > i):
                    arcs[(i,j)] = a
                    A_in[i, a] = 1
                    A_out[j, a] = 1
                    B[a, a] = t_max[i] - t_min[j] + T(i,j)
                    d[a] = t_max[i] - t_min[j]
                    cost[a] = profit_accept[i,j]
                    a += 1
                    
            if (possible[j,i] == 1):
                predecessors.append(j)
                if (j > i):
                    arcs[(j,i)] = a
                    A_in[j, a] = 1
                    A_out[i, a] = 1
                    B[a, a] = t_max[j] - t_min[i] + T(j,i)
                    d[a] = t_max[j] - t_min[i]
                    cost[a] = profit_accept[j,i]
                    a += 1
                
        s_list.append(successors)
        p_list.append(predecessors)
    
    A = A_in - A_out
    C = np.transpose(A[:n,])
    e = np.zeros(n+taxis+2)
    e[source] = 1
    e[sink] = -1
    
    # Maximum Flow Problem CVXPY
    x = cp.Variable(num_arcs, boolean = True)
    t = cp.Variable(n)
    
    ## Network Flow Constraints
    constraints = [A @ x == taxis * e]
    inflow = A_in @ x
    constraints += [inflow[:n+taxis] <= 1]
    ## Time Window Constraints
    constraints += [t_min[:n] <= t, t <= t_max[:n]]
    constraints += [B @ x + C @ t <= d]
    
    profit = np.transpose(cost) @ x
    objective = cp.Maximize(profit)
    problem = cp.Problem(objective, constraints)
    
    sims = 1
    run_time = 0
    for i in range(sims):
        problem.solve(verbose = True)
        run_time += problem.solver_stats.solve_time
#     run_times.append(run_time/sims)
#     objectives.append(objective.value)
    print("Time Window:", time_window)
    print("Run Time:", run_time/sims)
    print("Objective Value:", objective.value)
    
    x_val = spa.dok_matrix((n+taxis+2, n+taxis+2))
    for i in range(n+taxis+2):
        for j in s_list[i]:
            x_val[i,j] = x[arcs.get((i,j))].value
    print(x_val.tocoo())

# plt.figure(1)
# plt.plot(window_mins, run_times);
# plt.title("Run Times");
# plt.xlabel("Time Window");
# plt.ylabel("Run Time");
# plt.figure(2)
# plt.plot(window_mins, objectives);
# plt.title("Profit");
# plt.xlabel("Time Window");
# plt.ylabel("Profit");

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 20536 rows, 19930 columns and 118172 nonzeros
Model fingerprint: 0xd4e00d21
Variable types: 200 continuous, 19730 integer (19730 binary)
Coefficient statistics:
  Matrix range     [3e-02, 1e+07]
  Objective range  [1e-03, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -11.9512265
Presolve removed 20138 rows and 58 columns
Presolve time: 0.39s
Presolved: 398 rows, 19872 columns, 41062 nonzeros
Variable types: 0 continuous, 19872 integer (19872 binary)

Root relaxation: objective -8.285951e+02, 1801 iterations, 0.23 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0 

##  Taxi Paper Formulation With Time Window Constraints

In [6]:
window_mins = [0, 3]
time_windows = to_td(window_mins)
run_times = []
objectives = []

taxis = 2            # Number of taxis
taxi_lon = -73.9772  # Initial longitude of taxis
taxi_lat = 40.7527   # Initial latitude of taxis
mph = 20             # Taxi speed between passengers
driving_cost = .6    # in dollars per mile

possible = np.zeros((n+taxis, n+taxis))  # =1 if connection (i,j) is possible, =0 if impossible
profit_accept = np.zeros((n+taxis, n+taxis))   # Net profit from accepting passenger j (based on preceding passenger i)
                                         # Note: =0 if connection (i,j) is impossible
    
for time_window in time_windows:
    
    for i in range(n):
        for j in range(n):
            if (i == j): continue

            dist_between = haversine(data.loc[i,'dropoff_longitude'], data.loc[i, 'dropoff_latitude'],
                                          data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
            time_between = timedelta(hours = dist_between / mph)
            if (data.loc[i, 'dropoff_datetime'] + time_between > data.loc[j, 'pickup_datetime'] + time_window): continue
            else: possible[i,j] = 1
            profit_accept[i,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])

    for k in range(n, n+taxis):
        for j in range(n):

            dist_between = haversine(taxi_lon, taxi_lat, data.loc[j, 'pickup_longitude'], data.loc[j, 'pickup_latitude'])
            time_between = timedelta(hours = dist_between / mph)
            if (midnight + time_between > data.loc[j, 'pickup_datetime'] + time_window): continue
            else: possible[k,j] = 1
            profit_accept[k,j] = data.loc[j, 'total_amount'] - driving_cost * (dist_between + data.loc[j, 'trip_distance'])
    
    s_list = []
    p_list = []

    for i in range(n+taxis):

        successors = []
        predecessors = []

        for j in range(n+taxis):
            if (possible[i,j] == 1): successors.append(j)
            if (possible[j,i] == 1): predecessors.append(j)

        s_list.append(successors)
        p_list.append(predecessors)
        
    # Optimization
    p = {}
    x = {}
    t = {}
    t_min = {}
    t_max = {}
    for i in range(n):
        p[i] = cp.Variable(boolean = True)
        t[i] = cp.Variable()
        t_min[i] = to_minutes(data.loc[i, 'pickup_datetime'] - midnight)
        t_max[i] = to_minutes(data.loc[i, 'pickup_datetime'] - midnight + time_window)
    for i in range(n+taxis):
        for j in s_list[i]:
            x[i,j] = cp.Variable(boolean = True)

    constraints = []
    constraints += [cp.sum([x[i,j] for i in p_list[j]]) == p[j] for j in range(n)]
    constraints += [cp.sum([x[i,j] for j in s_list[i]]) <= p[i] for i in range(n)]
    constraints += [cp.sum([x[k,j] for j in s_list[k]]) <= 1 for k in range(n, n+taxis)]
    constraints += [t_min[i] <= t[i] for i in range(n)]
    constraints += [t[i] <= t_max[i] for i in range(n)]
    constraints += [t[j] - t[i] >= (t_min[j] - t_max[i]) + (T(i,j) - (t_min[j] - t_max[i])) * x[i,j]
                    for i in range(n) for j in s_list[i]]
    constraints += [t[j] >= t_min[j] + (T(k,j) - t_min[j]) * x[k,j]
                    for k in range(n, n+taxis) for j in s_list[k]]

    profit = cp.sum([profit_accept[i,j] * x[i,j] for i in range(n+taxis) for j in s_list[i]])
    objective = cp.Maximize(profit)
    problem = cp.Problem(objective, constraints)
    
    sims = 100
    run_time = 0
    for i in range(sims):
        problem.solve()
        run_time += problem.solver_stats.solve_time
#     run_times.append(run_time/sims)
#     objectives.append(objective.value)
    print("Time Window:", time_window)
    print("Run Time:", run_time/sims)
    print("Objective Value:", objective.value)
    
    x_val = spa.dok_matrix((n+taxis, n+taxis))
    for i in range(n+taxis):
        for j in s_list[i]:
            x_val[i,j] = x[i,j].value
    print(x_val.tocoo())

Time Window: 0:00:00
Run Time: 0.4022223663330078
Objective Value: 828.5950769919391
  (0, 2)	1.0
  (1, 4)	1.0
  (2, 3)	1.0
  (3, 8)	1.0
  (4, 5)	1.0
  (5, 6)	1.0
  (6, 10)	1.0
  (8, 9)	1.0
  (9, 11)	1.0
  (10, 16)	1.0
  (11, 12)	1.0
  (12, 13)	1.0
  (13, 15)	1.0
  (15, 24)	1.0
  (16, 25)	1.0
  (24, 31)	1.0
  (25, 28)	1.0
  (28, 34)	1.0
  (31, 35)	1.0
  (34, 36)	1.0
  (35, 37)	1.0
  (36, 42)	1.0
  (37, 41)	1.0
  (41, 44)	1.0
  (42, 46)	1.0
  :	:
  (121, 129)	1.0
  (129, 138)	1.0
  (131, 134)	1.0
  (134, 136)	1.0
  (136, 145)	1.0
  (138, 147)	1.0
  (145, 151)	1.0
  (147, 152)	1.0
  (151, 166)	1.0
  (152, 154)	1.0
  (154, 165)	1.0
  (165, 170)	1.0
  (166, 167)	1.0
  (167, 172)	1.0
  (170, 178)	1.0
  (172, 176)	1.0
  (176, 179)	1.0
  (178, 180)	1.0
  (179, 184)	1.0
  (180, 191)	1.0
  (184, 189)	1.0
  (189, 195)	1.0
  (195, 199)	1.0
  (200, 1)	1.0
  (201, 0)	1.0
828.5950769919391
Time Window: 0:03:00
Run Time: 0.31299686431884766
Objective Value: 899.8779009504111
  (0, 2)	1.0
  (1, 4)	1.0

## Map

In [ ]:
x_val = spa.dok_matrix((n+taxis+2, n+taxis+2))
for i in range(n+taxis+2):
    for j in s_list[i]:
        x_val[i,j] = x[arcs.get((i,j))].value
print("Time Window:", time_window)
print(x_val.tocoo())
print(objective.value)

In [ ]:
arcs = np.asarray(np.nonzero(x_val))
# locs = np.unique(arcs[arcs < 200])
# lons = data.iloc[locs, 3]
# lats = data.iloc[locs, 4]

loc_origin = arcs[0][:-taxis-2]
loc_destin = arcs[1][:-taxis-2]

lon_origin = data.iloc[loc_origin, 3].to_numpy()
lat_origin = data.iloc[loc_origin, 4].to_numpy()
lon_destin = data.iloc[loc_destin, 3].to_numpy()
lat_destin = data.iloc[loc_destin, 4].to_numpy()

In [ ]:
# from shapely.geometry import Point, LineString
from matplotlib.patches import FancyArrowPatch

nyc = gpd.read_file(gpd.datasets.get_path('nybb'))
# nyc = nyc.to_crs(epsg=3857)
nyc = nyc.to_crs(epsg=4326)
ax = nyc.plot(figsize=(20, 20), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax)

ax.set_xlim(-74.05, -73.85)
ax.set_ylim(40.68, 40.82)

# nodes = gpd.GeoDataFrame(geometry=gpd.points_from_xy(lons, lats))
# nodes.plot(color='green')

fig = plt.figure()

# One Taxi's Journey
taxi_num = n + 1
node = arcs[1][np.where(arcs[0] == taxi_num)] # First node (rider) for given taxi
while True:
    where = np.array(np.where(loc_origin == node))
    if (where.size == 0): break
    i = where[0][0]
    dx = lon_destin[i] - lon_origin[i]
    dy = lat_destin[i] - lat_origin[i]
    ax.arrow(lon_origin[i], lat_origin[i], dx, dy, color = 'pink', width=.0004)
    node = loc_destin[i]

plt.show()